In [ ]:
#!/usr/bin/env python3
'''
Main script
'''

import os
from pymatgen.io.vasp.inputs import Poscar

from CrySPY.interface import select_code
from CrySPY.job.ctrl_job import Ctrl_job
from CrySPY.IO import read_input as rin
from CrySPY.start import cryspy_init, cryspy_restart

from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version

from aiida.engine import run_get_node, run
from aiida.plugins import DataFactory, WorkflowFactory
import numpy as np
from aiida.orm import Code
from aiida.orm import Str, Dict, Int, List, Float
from aiida.engine import calcfunction, WorkChain, append_
from aiida.plugins import DataFactory
from itertools import cycle
import os
import io

from CrySPY.gen_struc.random.gen_pyxtal import Rnd_struc_gen_pyxtal

In [ ]:
import aiida
aiida.load_profile()

In [ ]:

# ## parallel submit
# ref.
# https://aiida.readthedocs.io/projects/aiida-core/en/latest/topics/workflows/usage.html?highlight=tocontext#submitting-sub-processes
#

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')
LammpsPotential = DataFactory('lammps.potential')
TrajectoryData = DataFactory('array.trajectory')


In [ ]:
rin.readin()          # read input data, cryspy,in
rin.algo

In [ ]:
rin.natot, rin.atype, rin.nat,rin.vol_factor, rin.vol_mu, rin.vol_sigma, rin.mindist, rin.spgnum, rin.symprec

In [ ]:
initial_structure_WorkChain = WorkflowFactory('cryspy.random_structures')

In [ ]:

params = {"vol_mu":rin.vol_mu, "vol_sigma":rin.vol_sigma, 
          "mindist":rin.mindist}

inputs = {"tot_struc": Int(rin.tot_struc),
    "natot":Int(rin.natot), "atype":List(list=rin.atype),
                                        "nat":List(list=rin.nat), "vol_factor":List(list=rin.vol_factor),
                                        "spgnum":Str(rin.spgnum), "symprec":Float(float(rin.symprec)),
         'params': Dict(dict=params)}
print("inputs", inputs)
initial_structures_result, node = run_get_node(initial_structure_WorkChain, **inputs)

In [ ]:
initial_structures = initial_structures_result["structures"]
initial_structures

In [ ]:
!verdi node graph generate $node.pk


In [ ]:

computer_local = 'localhost'
code_lammps_opt = get_or_create_code('lammps.optimize',
                                     computer_local, 'lammps')
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 4}
}


In [ ]:
code_lammps_opt.label

In [ ]:
# lammps potentials
pair_style = 'tersoff'
potential_dict = {
    'Ga Ga Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 1.0 1.44970 410.132 2.87 0.15 1.60916 535.199',
    'N  N  N': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 1.0 2.38426 423.769 2.20 0.20 3.55779 1044.77',
    'Ga Ga N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga N ': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 0.0 0.00000 0.00000 2.20 0.20 0.00000 0.00000',
    'N  N  Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 0.0 0.00000 0.00000 2.87 0.15 0.00000 0.00000'}
potential = DataFactory("lammps.potential")(
    type=pair_style, data=potential_dict
)
potential.attributes

In [ ]:
# lammps.optimize parameters
parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': [],
    'units': 'metal',
    'relax': {
        'type': 'aniso',
        'pressure': 0.0,
        'vmax': 0.005,
    },
    "minimize": {
        'style': 'cg',
        'energy_tolerance': 1.0e-5,
        'force_tolerance': 1.0e-3,
        'max_iterations': 1000,
        'max_evaluations': 10000
        }
})


In [ ]:

optimization_simulator_lammps_WorkChain = \
WorkflowFactory('cryspy.optimize_structures_lammps')

inputs = {'code_string': Str(code_lammps_opt.label),
         "initial_structures": initial_structures,
         "potential": potential,
         "parameters": parameters}

results, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)


In [ ]:
import pandas as pd
df = pd.DataFrame(results["results"].get_dict())
df

In [ ]:
!verdi node graph generate $node.pk
